## Evaluation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
import livecell_tracker.core.datasets
from livecell_tracker.segment.detectron_utils import gen_cfg

dir_path = Path('../cxa-data/june_2022_data_8bit_png/day0_Notreat_Group 1_wellA1_RI_MIP_stitched')
# imgs = livecell_tracker.core.datasets.LiveCellImageDataset(dir_path, ext="tif")
imgs = livecell_tracker.core.datasets.LiveCellImageDataset(dir_path, ext="png")
model_train_dir = os.path.join("./notebook_results/train_log/", "detectron_train_output__ver0.0.2")

120 png img file paths loaded: 


In [2]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

In [3]:
from detectron2.data.datasets import register_coco_instances

train_data_path = r"""."""
train_json_path = r"""./notebook_results/labelme2coco_test_output_8bit_png__ver0.0.2/train.json"""

val_data_path = r"""."""
val_json_path = r"""./notebook_results/labelme2coco_test_output_8bit_png__ver0.0.2/val.json"""

register_coco_instances("deepfashion_train", {}, train_json_path, train_data_path)
register_coco_instances("deepfashion_val", {}, val_json_path, val_data_path)
register_coco_instances("nanolive_train_raw", {}, r"./notebook_results/labelme2coco_test_output/train.json", ".")

In [4]:
import detectron2
# len(detectron2.data.DatasetCatalog.get("deepfashion_train"))
# len(detectron2.data.DatasetCatalog.get("deepfashion_val"))
# detectron2.data.MetadataCatalog.get("nanolive_train_raw")
len(detectron2.data.DatasetCatalog.get("nanolive_train_raw"))


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



56

In [5]:
def detectron_predict_single_img(img):
    outputs = predictor(img)
    return outputs

def detectron_predict_and_visualize_img(img):
    outputs = detectron_predict_single_img(img)
    v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    figure = plt.figure(figsize=(20, 20), dpi=80)
    plt.imshow(v.get_image()[:, :, ::-1])

In [6]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
#Call the COCO Evaluator function and pass the Validation Dataset
def eval_by_detectron(model, eval_dataset_name: str, cfg):
    evaluator = COCOEvaluator(eval_dataset_name, cfg, False, output_dir="./output/")
    val_loader = build_detection_test_loader(cfg, eval_dataset_name)
    #Use the created predicted model in the previous step
    return inference_on_dataset(model, val_loader, evaluator)

### detectron model3 on updated annotation results

In [7]:
cfg = gen_cfg(output_dir=r"./notebook_results/train_log/detectron_run3_output")
predictor = DefaultPredictor(cfg)
eval_dataset_name = "nanolive_train_raw"
# eval_dataset_name = "nanolive_train_raw"
model = predictor.model
eval_by_detectron(model, eval_dataset_name, cfg)

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.

Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

c:\Users\dummy\Anaconda3\envs\cv-general\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
No predictions from the model!


OrderedDict([('bbox',
              {'AP': nan,
               'AP50': nan,
               'AP75': nan,
               'APs': nan,
               'APm': nan,
               'APl': nan})])

In [8]:
cfg = gen_cfg(output_dir=r"./notebook_results/train_log/detectron_run3_output")
predictor = DefaultPredictor(cfg)
eval_dataset_name = "deepfashion_train"
# eval_dataset_name = "nanolive_train_raw"
model = predictor.model
eval_by_detectron(model, eval_dataset_name, cfg)

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.

Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.

OrderedDict([('bbox',
              {'AP': 0.0001712974065572647,
               'AP50': 0.0008564870327863236,
               'AP75': 0.0,
               'APs': 0.0,
               'APm': 0.00017554946984060107,
               'APl': 0.0,
               'AP-nucleus': 0.0,
               'AP-cellbody-with-protrusion': 0.0006851896262290588,
               'AP-apoptotic-cell': 0.0,
               'AP-mitotic-cell': 0.0}),
             ('segm',
              {'AP': 0.0005617884154806049,
               'AP50': 0.0024201876440733544,
               'AP75': 0.0,
               'APs': 0.0,
               'APm': 0.006967042858131969,
               'APl': 0.0,
               'AP-nucleus': 0.0005341795963497728,
               'AP-cellbody-with-protrusion': 0.0017129740655726473,
               'AP-apoptotic-cell': 0.0,
               'AP-mitotic-cell': 0.0})])

### latest detectron model on updated annotation results

In [9]:
cfg = gen_cfg(output_dir=model_train_dir)
predictor = DefaultPredictor(cfg)

eval_dataset_name = "deepfashion_train"
model = predictor.model
eval_by_detectron(model, eval_dataset_name, cfg)

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.

Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.266
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.012
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.101
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.204
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.013
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.107
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.170
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.010
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.113
 Average Recall     (AR) @[ IoU=0.50:0.

OrderedDict([('bbox',
              {'AP': 13.959243502448931,
               'AP50': 26.55613606493517,
               'AP75': 13.064345165621564,
               'APs': 1.188118811881188,
               'APm': 10.067298117285146,
               'APl': 20.37967434917972,
               'AP-nucleus': 11.38204314694953,
               'AP-cellbody-with-protrusion': 44.454930862846204,
               'AP-apoptotic-cell': 0.0,
               'AP-mitotic-cell': 0.0}),
             ('segm',
              {'AP': 12.8516977504464,
               'AP50': 24.97970669054998,
               'AP75': 12.233597699190009,
               'APs': 1.188118811881188,
               'APm': 8.776137977672734,
               'APl': 22.15411609400595,
               'AP-nucleus': 9.560601362158778,
               'AP-cellbody-with-protrusion': 41.84618963962681,
               'AP-apoptotic-cell': 0.0,
               'AP-mitotic-cell': 0.0})])